In [1]:
import numpy as np
import sys
sys.path.append('./Group_SVM')
sys.path.append('../SPGL1_python_port')

from Our_methods import use_FO1_CG, use_FO2_CG, use_RP_CG
from Benchmarks import use_Gurobi, use_SCS
# from simulate_data_classification import *
from simulate_data_group import simulate_data_group

In [2]:
# generate data
## 'trash.txt' is a temperory txt file to save some of the output

N, P = 100, 1000
type_Sigma, k0, rho, tau_SNR = 2, 10, 0.2, 0.5
f = open('trash.txt', 'w')
seed_X = 111
feat_by_group = 10
num_groups = int(P/feat_by_group)
group_to_feat = np.array([range(10*k, 10*(k+1)) for k in range(num_groups)]) 
X_train, l2_X_train, y_train, u_positive = simulate_data_group(type_Sigma, N, P, 
                                                                       group_to_feat, k0, rho, 
                                                                       tau_SNR, seed_X, f)
aux          = np.sum( np.abs(X_train), axis=0)
aux          = [np.sum(aux[idx]) for idx in group_to_feat]
alpha_max    = np.max(aux)  
lam = rho* alpha_max

(100, 1000)

DATA CREATED for N=100, P=1000, k0=10 Rho=0.2 Sigma=2 Seed_X=111


In [3]:
## Use column generation LP inicialized with first order method. See the paper for more details.
## The input "tol" is the tolerance parameter for searching reduced cost (the parameter "epsilon" in the paper)


obj, time_total, time_CG, beta, beta0 = use_FO2_CG(X_train, y_train, lam, group_to_feat, tol=0.01)



Alpha : 81.62066756035144
0
Test CV
4.1029759517993813e-17

Number of iterations: 1
Len group support smoothing: 0
Convergence rate           : 0.0
Time smoothing: 0.009

Alpha : 32.64826702414057

Alpha : 16.324133512070286

Number of iterations                       : 0
Total time smoothing for hinge: 0.012
Number of constraints: 100
Number of groups     : 0
Using license file /home/software/gurobi/gurobi.lic
Set parameter TokenServer to value engaging-license-001
Round 1
Time optimizing = 0.015319585800170898
Number of groups added: 10
Round 2
Time optimizing = 0.13754630088806152

TIME CG = 0.1402266025543213

Obj value   = 47.35474457027614
Len support = 100
b0   = -0.01629275098819607

Number violated constraints =  48
Len support dual = 54


In [4]:
## Use Gurobi
obj, runtime, beta, beta0 = use_Gurobi(X_train, y_train, lam, group_to_feat)


Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 3200 rows, 2201 columns and 106300 nonzeros
Model fingerprint: 0xe31759b6
Coefficient statistics:
  Matrix range     [7e-07, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 1100 rows and 1000 columns
Presolve time: 0.08s
Presolved: 2100 rows, 1201 columns, 104200 nonzeros

Ordering time: 0.01s

Barrier statistics:
 Free vars  : 1101
 AA' NZ     : 2.240e+05
 Factor NZ  : 2.260e+05 (roughly 3 MBytes of memory)
 Factor Ops : 2.483e+07 (less than 1 second per iteration)
 Threads    : 26

Barrier performed 0 iterations in 0.26 seconds
Barrier solve interrupted - model solved by another algorith

In [5]:
## Use SCS
obj, runtime, beta, beta0 = use_SCS(X_train, y_train, lam, group_to_feat)


----------------------------------------------------------------------------
	SCS v2.1.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 106300
eps = 1.00e-04, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 2201, constraints m = 3200
Cones:	linear vars: 3200
Setup time: 3.51e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.90e+00  3.52e+00  9.99e-01 -5.85e+02  6.40e+02  0.00e+00  4.78e-03 
   100| 8.10e-04  2.14e-03  1.37e-03  4.75e+01  4.73e+01  3.75e-15  1.52e-01 
   200| 4.49e-04  1.26e-03  8.32e-04  4.74e+01  4.73e+01  2.43e-14  2.78e-01 
   300| 4.62e-04  1.02e-03  3.99e-05  4.73